# IMDB DATA SET

In [11]:
pip install --user your-package-name

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install --default-timeout=100 tensor2tensor

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install --default-timeout=100 tensorflow matplotlib

Note: you may need to restart the kernel to use updated packages.


In [14]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os

# Other setup
Modes = tf.estimator.ModeKeys

# Setting up some directories
data_dir = os.path.expanduser("~/t2t/data")
tmp_dir = os.path.expanduser("~/t2t/tmp")
train_dir = os.path.expanduser("~/t2t/train")
checkpoint_dir = os.path.expanduser("~/t2t/checkpoints")
os.makedirs(data_dir, exist_ok=True)
os.makedirs(tmp_dir, exist_ok=True)
os.makedirs(train_dir, exist_ok=True)
os.makedirs(checkpoint_dir, exist_ok=True)


In [15]:
import tensorflow_datasets as tfds

# List of all datasets
print(tfds.list_builders())


['abstract_reasoning', 'accentdb', 'aeslc', 'aflw2k3d', 'ag_news_subset', 'ai2_arc', 'ai2_arc_with_ir', 'amazon_us_reviews', 'anli', 'answer_equivalence', 'arc', 'asqa', 'asset', 'assin2', 'bair_robot_pushing_small', 'bccd', 'beans', 'bee_dataset', 'beir', 'big_patent', 'bigearthnet', 'billsum', 'binarized_mnist', 'binary_alpha_digits', 'ble_wind_field', 'blimp', 'booksum', 'bool_q', 'bucc', 'c4', 'c4_wsrs', 'caltech101', 'caltech_birds2010', 'caltech_birds2011', 'cardiotox', 'cars196', 'cassava', 'cats_vs_dogs', 'celeb_a', 'celeb_a_hq', 'cfq', 'cherry_blossoms', 'chexpert', 'cifar10', 'cifar100', 'cifar100_n', 'cifar10_1', 'cifar10_corrupted', 'cifar10_n', 'citrus_leaves', 'cityscapes', 'civil_comments', 'clevr', 'clic', 'clinc_oos', 'cmaterdb', 'cnn_dailymail', 'coco', 'coco_captions', 'coil100', 'colorectal_histology', 'colorectal_histology_large', 'common_voice', 'conll2002', 'conll2003', 'controlled_noisy_web_labels', 'coqa', 'cos_e', 'cosmos_qa', 'covid19', 'covid19sum', 'crema_d

In [16]:
import tensorflow_datasets as tfds

# IMDB dataset 
dataset, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']


In [17]:
# Info about the dataset
print('Dataset Info:', info)

Dataset Info: tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/plain_text/1.0.0',
    description="""
    Large Movie Review Dataset. This is a dataset for binary sentiment
    classification containing substantially more data than previous benchmark
    datasets. We provide a set of 25,000 highly polar movie reviews for training,
    and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Plain text
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_path='/Users/apple/tensorflow_datasets/imdb_reviews/plain_text/1.0.0',
    file_format=tfrecord,
    download_size=80.23 MiB,
    dataset_size=129.83 MiB,
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
        'text': Text(shape=(), dtype=string),
    }),
    supervised_keys=('text', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=2500

In [18]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

# Load the IMDB Reviews dataset
dataset, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

# Split the dataset into train and test sets
train_dataset, test_dataset = dataset['train'], dataset['test']

# Example from the training dataset
for example_text, example_label in train_dataset.take(1):
    print("Text:", example_text.numpy())
    print("Label:", example_label.numpy())


Text: b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
Label: 0


# Sentiment Analysis Using PreTrained Model

In [3]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Load the IMDB dataset
dataset, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

# Prepare the data
train_sentences = []
train_labels = []

test_sentences = []
test_labels = []

# Extract sentences and labels from the training dataset
for s, l in train_dataset:
    train_sentences.append(str(s.numpy()))
    train_labels.append(l.numpy())

# Extract sentences and labels from the testing dataset
for s, l in test_dataset:
    test_sentences.append(str(s.numpy()))
    test_labels.append(l.numpy())

# Convert labels to numpy array
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

# Tokenize and pad the sentences
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_sentences)

train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, maxlen=120, truncating='post', padding='post')

test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, maxlen=120, truncating='post', padding='post')


In [4]:
import tensorflow as tf

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=16, input_length=120),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 120, 16)           160000    
                                                                 
 global_average_pooling1d_1  (None, 16)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_4 (Dense)             (None, 16)                272       
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160289 (626.13 KB)
Trainable params: 160289 (626.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
history = model.fit(train_padded, train_labels,
                    epochs=10,
                    validation_data=(test_padded, test_labels))


Epoch 1/10
782/782 [==============================] - 5s 5ms/step - loss: 0.5264 - accuracy: 0.7623 - val_loss: 0.3925 - val_accuracy: 0.8270
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.3136 - accuracy: 0.8695 - val_loss: 0.3739 - val_accuracy: 0.8341
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2572 - accuracy: 0.8982 - val_loss: 0.3851 - val_accuracy: 0.8314
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2221 - accuracy: 0.9146 - val_loss: 0.4127 - val_accuracy: 0.8257
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1962 - accuracy: 0.9267 - val_loss: 0.4470 - val_accuracy: 0.8188
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1757 - accuracy: 0.9357 - val_loss: 0.4956 - val_accuracy: 0.8101
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1589 - accuracy: 0.9433 - val_loss: 0.5193 - val_accuracy: 0.8127
Epoch 

In [6]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_padded, test_labels)

print(f"Test accuracy: {test_accuracy}")


782/782 [==============================] - 2s 2ms/step - loss: 0.6656 - accuracy: 0.7976
Test accuracy: 0.7976400256156921


# BERT-BASE-UNCEASED

In [ ]:
!pip install tensorflow
!pip install transformers
!pip install datasets

In [1]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from datasets import load_dataset
from sklearn.model_selection import train_test_split

In [2]:
# Load the IMDb dataset
dataset = load_dataset('imdb')

# Split the dataset into training and testing
train_dataset = dataset['train']
test_dataset = dataset['test']

In [3]:
print(train_dataset[0])

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [4]:
print(train_dataset.features)

{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}


In [5]:
print(train_dataset.column_names)

['text', 'label']


In [6]:
!pip install torch torchvision

In [7]:
pip install torch torchvision torchaudio

Note: you may need to restart the kernel to use updated packages.


In [8]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Tokenization function
def encode_reviews(data):
    return tokenizer(data['text'], truncation=True, padding='max_length', max_length=512)

# Encode the input data
train_dataset = train_dataset.map(encode_reviews, batched=True)
test_dataset = test_dataset.map(encode_reviews, batched=True)

# Set format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


In [9]:
from torch.utils.data import DataLoader

# Create the DataLoaders for our training and validation sets.
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8)
validation_dataloader = DataLoader(test_dataset, batch_size=8)


In [20]:
from transformers import AutoModelForSequenceClassification
import torch
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top.
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 2, # The number of output labels--2 for binary classification.  
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
# run this model on the CPU.
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS (GPU) is available. Using GPU...")
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU instead.")
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


MPS (GPU) is available. Using GPU...


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [21]:
from transformers import AdamW, get_linear_schedule_with_warmup

# AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

/Users/apple/anaconda3/envs/mlp/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [22]:
import datetime
import time

# Helper function to format elapsed times as hh:mm:ss
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [23]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8
                )

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we could also have tried for more epochs.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value
                                            num_training_steps = total_steps)


/Users/apple/anaconda3/envs/mlp/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [26]:
import numpy as np
from sklearn.metrics import f1_score
import random
import torch
import time

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# Seed value for reproducibility
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# we set our device to CPU.
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS (GPU) is available. Using GPU...")
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU instead.")
print(f'Using device: {device}')

# Move the model to the CPU
model.to(device)

# Store the average loss after each epoch so we can plot them.
loss_values = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# For each epoch...
for epoch_i in range(0, epochs):
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. 
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        b_input_ids = batch['input_ids'].to(device)
        b_input_mask = batch['attention_mask'].to(device)
        b_labels = batch['label'].to(device)

        # clear any previously calculated gradients before performing a
        # backward pass. 
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs.loss

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. 
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure performance on
    # validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack the inputs from our dataloader
        b_input_ids = batch['input_ids'].to(device)
        b_input_mask = batch['attention_mask'].to(device)
        b_labels = batch['label'].to(device)
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs.logits

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


MPS (GPU) is available. Using GPU...
Using device: mps

======== Epoch 1 / 4 ========
Training...
  Batch    40  of  3,125.    Elapsed: 0:00:49.
  Batch    80  of  3,125.    Elapsed: 0:01:36.
  Batch   120  of  3,125.    Elapsed: 0:02:24.
  Batch   160  of  3,125.    Elapsed: 0:03:11.
  Batch   200  of  3,125.    Elapsed: 0:03:59.
  Batch   240  of  3,125.    Elapsed: 0:04:46.
  Batch   280  of  3,125.    Elapsed: 0:05:33.
  Batch   320  of  3,125.    Elapsed: 0:06:20.
  Batch   360  of  3,125.    Elapsed: 0:07:07.
  Batch   400  of  3,125.    Elapsed: 0:07:54.
  Batch   440  of  3,125.    Elapsed: 0:08:42.
  Batch   480  of  3,125.    Elapsed: 0:09:29.
  Batch   520  of  3,125.    Elapsed: 0:10:16.
  Batch   560  of  3,125.    Elapsed: 0:11:03.
  Batch   600  of  3,125.    Elapsed: 0:11:50.
  Batch   640  of  3,125.    Elapsed: 0:12:37.
  Batch   680  of  3,125.    Elapsed: 0:13:24.
  Batch   720  of  3,125.    Elapsed: 0:14:12.
  Batch   760  of  3,125.    Elapsed: 0:14:59.
  Batch  

  Batch   400  of  3,125.    Elapsed: 0:07:51.
  Batch   440  of  3,125.    Elapsed: 0:08:38.
  Batch   480  of  3,125.    Elapsed: 0:09:25.
  Batch   520  of  3,125.    Elapsed: 0:10:12.
  Batch   560  of  3,125.    Elapsed: 0:10:59.
  Batch   600  of  3,125.    Elapsed: 0:11:46.
  Batch   640  of  3,125.    Elapsed: 0:12:34.
  Batch   680  of  3,125.    Elapsed: 0:13:21.
  Batch   720  of  3,125.    Elapsed: 0:14:08.
  Batch   760  of  3,125.    Elapsed: 0:14:55.
  Batch   800  of  3,125.    Elapsed: 0:15:42.
  Batch   840  of  3,125.    Elapsed: 0:16:29.
  Batch   880  of  3,125.    Elapsed: 0:17:17.
  Batch   920  of  3,125.    Elapsed: 0:18:04.
  Batch   960  of  3,125.    Elapsed: 0:18:51.
  Batch 1,000  of  3,125.    Elapsed: 0:19:38.
  Batch 1,040  of  3,125.    Elapsed: 0:20:25.
  Batch 1,080  of  3,125.    Elapsed: 0:21:11.
  Batch 1,120  of  3,125.    Elapsed: 0:21:58.
  Batch 1,160  of  3,125.    Elapsed: 0:22:45.
  Batch 1,200  of  3,125.    Elapsed: 0:23:32.
  Batch 1,240

# Attention

In [1]:
pip install tensorflow --timeout=100

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load the IMDB dataset
(train_data, test_data), info = tfds.load('imdb_reviews', split=['train', 'test'], with_info=True, as_supervised=True)

# Preprocess the data
BUFFER_SIZE = 10000
BATCH_SIZE = 64

def preprocess_text(text, label):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, "<br />", " ")
    text = tf.strings.regex_replace(text, "[^a-zA-Z']", " ")
    return text, label

train_data = train_data.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).map(preprocess_text)
test_data = test_data.batch(BATCH_SIZE).map(preprocess_text)

# Text Vectorization
VOCAB_SIZE = 10000
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
# Adapt the encoder on the train data
encoder.adapt(train_data.map(lambda text, label: text))

# Transformer Model
def transformer_encoder(inputs, embed_dim, num_heads, ff_dim, rate=0.1):
    # Attention and Normalization
    attention_output = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(inputs, inputs)
    attention_output = tf.keras.layers.Dropout(rate)(attention_output)
    attention_output = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs + attention_output)

    # Feed Forward Network
    ff_output = tf.keras.layers.Dense(ff_dim, activation="relu")(attention_output)
    ff_output = tf.keras.layers.Dense(embed_dim)(ff_output)
    ff_output = tf.keras.layers.Dropout(rate)(ff_output)
    ff_output = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention_output + ff_output)
    return ff_output

# Model Creation
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network

inputs = tf.keras.layers.Input(shape=(), dtype=tf.string)
x = encoder(inputs)
x = tf.keras.layers.Embedding(input_dim=len(encoder.get_vocabulary()), output_dim=embed_dim)(x)
x = transformer_encoder(x, embed_dim, num_heads, ff_dim)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dense(20, activation="relu")(x)
x = tf.keras.layers.Dropout(0.1)(x)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_data, epochs=2, validation_data=test_data)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_data)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)


Epoch 1/2
391/391 [==============================] - 1348s 3s/step - loss: 0.4966 - accuracy: 0.7270 - val_loss: 0.2798 - val_accuracy: 0.8876
Epoch 2/2
391/391 [==============================] - 362s 926ms/step - loss: 0.2762 - accuracy: 0.8872
Test Loss: 0.2762269377708435
Test Accuracy: 0.8871999979019165
